In [1]:
## THIS TEST REQUIRES ROOT PERMISSION FOR JUPYTER
# $ jupyter notebook --allow-root

## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hi'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
alias testuser="sed -r "s/"$USER"+/user/g""
alias testnum="sed -r "s/[0-9]/X/g"" 

In [5]:
# Setting a temp directory for tests
TMP=/tmp/test-admin-commands

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..

alias | wc -l

2


In [6]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 3245)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-3245

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-admin-commands/test-3245


In [7]:
# Count aliases proper
alias | wc -l
# Count functions
typeset -f | egrep '^\w+' | wc -l

3
0


In [8]:
# TEST 1 - SOFTWARE CONTROL ALIASES
# NOTE: sudo is revoked from each alias (e.g. alias apt-uninstall = "sudo apt-get remove")

alias apt-install='apt-get install --yes --fix-missing --no-remove'
alias apt-search='apt-cache search'
alias apt-installed='apt list --installed'
alias apt-uninstall='apt-get remove'
alias dpkg-install='dpkg --install '

In [9]:
## 1A) apt-installed : VIEWS ALL INSTALLED PACKAGES
# Note: Entire list of the package is very large.

# Example: Sorts and select all packages related to Python3
apt-installed *python3* | grep ipython | cut -d \/ -f 1 | testnum



ipythonX
pythonX-ipython-genutils
pythonX-ipython


In [10]:
## 1B) apt-search : SEARCHES DESIRED PACKAGES IN THE apt REPO

# Example: Searching for rolldice, a simple apt package
apt-search rolldice

rolldice - virtual dice roller


In [11]:
## 1C) apt-install : INSTALLS THE PACKAGE IF PRESENT IN THE REPO
## NOTE: Installing packages requires sudo

# Example: Installing rolldice (need a more suitable example - but works)

# | $ apt-install rolldice
# | E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
# | E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?

In [12]:
## 1D) apt-uninstall : UNINSTALLS/REMOVES DESIRED PACKAGE IF PRESENT
## NOTE: Uninstalling packages requires sudo

# # Example: Uninstalling rolldice

# $ apt-uninstall rolldice
# | E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
# | E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [13]:
## 1E) dpkg-install : INSTALLS A DEBIAN PACKAGE
## NOTE: dpkg-install also requires sudo

# Example: Uninstalling rolldice (requires superuser - but works)

# $ dpkg-install rolldice
# | dpkg: error: requested operation requires superuser privilege

In [14]:
## TEST 2 - SYSTEM OPERATION ALIASES
alias restart-network='sudo ifdown eth0; sudo ifup eth0'
alias hibernate-system='sudo systemctl hibernate'
alias suspend-system='sudo systemctl suspend'
alias blank-screen='xset dpms force off'
alias stop-service='systemctl stop'
alias restart-service='sudo systemctl restart'
# stop-service IS ONLY SAFE TO USE FOR TEST FOR SOME SERVICES

# SOME ALIASES: ADDED BY ME
alias start-service='systemctl start'
alias list-all-service='systemctl --type=service'
alias restart-service-sudoless='systemctl restart'
alias status-service='systemctl status'
alias service-status='status-service'

In [15]:
# 2A) list-all-service : LISTS ALL SERVICES ON THE MACHINE (ALL STATES)
list-all-service | grep ufw

  ufw.service                                           loaded active exited  Uncomplicated firewall


In [16]:
# # 2B) stop-service : STOPS DESIRED SERVICE (password prompt appears; p/w required to start/stop the service)
# stop-service unattended-upgrades
# # service-status OR status-service VIEWS STATUS OF THE SERVICE
# status-service unattended-upgrades 

# # commented - tests halts after completon

In [17]:
# 2C) start-service : STARTS DESIRED SERVICE (password prompt appears; p/w required to start/stop the service)
start-service unattended-upgrades

# service-status OR status-service VIEWS STATUS OF THE SERVICE
service-status unattended-upgrades | head -n 1

● unattended-upgrades.service - Unattended Upgrades Shutdown


In [18]:
# 2D) restart-service-sudoless : SUDO-LESS METHOD TO RESTART DESIRED SERVICE 
# NOTE: password prompt appears; p/w required to start/stop the service 
restart-service-sudoless unattended-upgrades

# service-status OR status-service VIEWS STATUS OF THE SERVICE
service-status unattended-upgrades | grep Active | cut -d \s -f 1

# requires password but tests completes after (check need for batspp)

     Active: active (running) 


In [19]:
## TEST 3 - GET FREE FILENAME

# get-free-filename(base, [sep=""]): get filename starting with BASE that is not used.
# Notes: 1. If <base> exists <base><sep><N> checked until the filename not used (for N in 2, 3, ... ).
# 2. See sudo-admin for sample usage; also see rename-with-file-date.
#
function get-free-filename() {
    local base="$1"
    local sep="$2"
    local L=1
    local filename="$base"
    ## DEBUG: local -p
    while [ -e "$filename" ]; do
        let L++
        filename="$base$sep$L"
    done;
    ## DEBUG: local -p
    echo "$filename"
}

In [20]:
## 3A) get-free-filename : get filename starting with BASE that is not used
printf "HELLO THERE,\nI AM EXTREMELY PLEASED TO USE UBUNTU." > abc.txt

get-free-filename "abc.txt" 2      #DOUBT(?)
linebr
ls -l | awk '!($6="")' | testuser | testnum
linebr
cat "abc.txt"


abc.txt22
--------------------------------------------------------------------------------
total X    
-rw-rw-r-- X user user XX  XX XX:XX abc.txt
--------------------------------------------------------------------------------
HELLO THERE,
I AM EXTREMELY PLEASED TO USE UBUNTU.

In [21]:
# ## TEST 4 - SUDO ADMIN

# # sudo-admin(): create typescript as sudo user using filename based on current
# # date using numeric suffixes if necessary until the filename is free.
# # note: exsting _config*.log files are made read-only so not later overwritten
# # by accident
# function sudo-admin () {
#     local prefix="_config-"
#     local base="$prefix$(todays-date).log"
#     sudo chmod ugo-w "$prefix"*.log*
#     local script_log=$(get-free-filename "$base")
#     sudo --set-home   script --flush "$script_log"
# }